In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/codes/EnergyForecastingCompetition

/content/drive/MyDrive/codes/EnergyForecastingCompetition


In [ ]:
%ls

 comp_files/                          Forecasting.ipynb
'Copy of Model-Buildings.ipynb'       Model-Buildings.ipynb
'Copy of SolarPanel05-Minoli.ipynb'   Model.ipynb
 csvfile.csv                         'Model- Mudith.ipynb'
 csvfile.gsheet                      'Monthly Building Data.gdoc'
 data_loader.py                       phase_1_data.tsf
 data_loader.py.1                     __pycache__/
 data_loader.py.2                     SolarPanel05-Minoli.ipynb
'Dataset Analysis.gslides'            Stacking.ipynb
 ERA5_Weather_Data_Monash.csv         sub.csv


## **Importing Libraries**

In [ ]:
from datetime import datetime
from numpy import distutils
from sklearn.model_selection import train_test_split
import csv
import distutils
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import lightgbm as lgbm
import random

## **Loading the Dataset**

In [ ]:
from datetime import datetime
from numpy import distutils
from sklearn.model_selection import train_test_split
import csv
import distutils
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#from prophet import Prophet


# Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(full_file_path_and_name, replace_missing_vals_with = 'NaN', value_column_name = "series_value"):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, 'r', encoding='cp1252') as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"): # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (len(line_content) != 3):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if len(line_content) != 2:  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                    
                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(distutils.util.strtobool(line_content[1]))
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(distutils.util.strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception("Missing attribute section. Attribute section must come before data.")

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception("Missing attribute section. Attribute section must come before data.")
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if(len(series) == 0):
                            raise Exception("A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol")

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if (numeric_series.count(replace_missing_vals_with) == len(numeric_series)):
                            raise Exception("All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series.")

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(full_info[i], '%Y-%m-%d %H-%M-%S')
                            else:
                                raise Exception("Invalid attribute type.") # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if(att_val == None):
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        
        loaded_data = pd.DataFrame(all_data)
        #print(loaded_data.iloc[0,2])
        return loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length



In [ ]:
loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("phase_1_data.tsf",replace_missing_vals_with=np.nan)

In [ ]:
def getTimeBin(curr_time):#return the time index value of a day, when given the time of timestamp
  return curr_time.hour*4 + curr_time.minute//15

**Adding Features to the Input**

In [ ]:
def addNoiseToFeatures(features,noOfNoises):
  f_N= len(features)
  #add to noises
  for i in range(noOfNoises):
    noisePos= random.randint(0,f_N-1)
    features[noisePos]=random.uniform(0.0,2)*features[noisePos]



## **Build the light GBM Model**

Setting the parameters
 https://lightgbm.readthedocs.io/en/latest/Parameters.html#parameters-format

In [ ]:

params = {
        'nthread': 10,
         'max_depth': 5,
#         'max_depth': 9,
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': 'mape', # this is abs(a-e)/max(1,a)
#         'num_leaves': 39,
        'num_leaves': 64,
        'learning_rate': 0.075,
       'feature_fraction': 0.9,
#         'feature_fraction': 0.8108472661400657,
#         'bagging_fraction': 0.9837558288375402,
       'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'lambda_l1': 3.097758978478437,
        'lambda_l2': 2.9482537987198496,
#       'lambda_l1': 0.06,
#       'lambda_l2': 0.1,
        'verbose': 1,
        'min_child_weight': 6.996211413900573,
        'min_split_gain': 0.037310344962162616,
 }
# """
# params = {
#           "objective" : "poisson",
#           "metric" :"rmse",
#           "force_row_wise" : True,
#           "learning_rate" : 0.075,
#           "sub_row" : 0.75,
#           "bagging_freq" : 1,
#           "lambda_l2" : 0.1,
#           "metric": ["rmse"],
#           'verbosity': 1,
#           'num_iterations' : 1200,
#           'num_leaves': 128,
#           "min_data_in_leaf": 100,
#          }

**Defining the model**

https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.train.html#lightgbm.train

lightgbm.train(params, train_set, num_boost_round=100, valid_sets=None, valid_names=None, fobj=None, feval=None, init_model=None, feature_name='auto', categorical_feature='auto', early_stopping_rounds=None, evals_result=None, verbose_eval='warn', learning_rates=None, keep_training_booster=False, callbacks=None)


In [ ]:
def createDatasets(building_number, lag, start_index=0, number_of_days_to_predict=31):

  # Extract Data
  building_series = loaded_data.iloc[building_number]['series_value'][start_index:].to_numpy()
  
  times = pd.date_range(loaded_data.iloc[building_number]['start_timestamp'], periods=len(loaded_data.iloc[building_number]['series_value']), freq='15min')
  times=times[start_index:]

  # Treating null values
  count=0
  for i,bv in enumerate(building_series):
    if(pd.isna(bv)): 
      count+=1
      building_series[i]=building_series[i-96] # values of previous day

  train_data_x = []
  train_data_y = []
  

  for i in range(building_series.shape[0]-lag):
    features = list(building_series[i:i+lag])
    
    window_mean = np.mean(building_series[i:i+lag])
    window_std = np.std(building_series[i:i+lag])

  # features.append(getTimeBin(times[i+lag]))
  # features.append(times[i+lag].hour)
  # features.append(times[i+lag].dayofweek)
  # features.append(times[i+lag].month)
  # features.append(window_mean)
  # features.append(window_std)

    month=times[i+lag].month

    train_data_x.append(np.array(features))
    train_data_y.append(building_series[i+lag])

  X = np.array(train_data_x)
  y = np.array(train_data_y)

  X_normed=X/X.mean(axis=0)
  y_normed=y/y.mean(axis=0)
  y_mn=y.mean(axis=0)

  # print(y_mn)
  # print(X_normed.shape)
  # print(y_normed.shape)

  X_train, X_val, y_train, y_val = train_test_split(X_normed, y_normed, test_size=0.2, shuffle=True) # 0.25 x 0.8 = 0.2

  lgb_training_data = lgbm.Dataset(X_train,label=y_train) 
  lgb_valid_data = lgbm.Dataset(X_val,label=y_val,reference=lgb_training_data)

  return lgb_training_data, lgb_valid_data, train_data_x, train_data_y, X_normed, y_normed, y_mn, times


In [ ]:
def createSolarDatasets(raw_number, lag, start_index=0, number_of_days_to_predict=31):
  solar_panel = loaded_data.iloc[raw_number]['series_value'].to_numpy()
  times = pd.date_range(loaded_data.iloc[raw_number]['start_timestamp'], periods=len(loaded_data.iloc[raw_number]['series_value']), freq='15min')
  weather_df = pd.read_csv("ERA5_Weather_Data_Monash.csv")
  #weather_df = weather_df[82622:]   #2019-06-05 14:00:00 onwards

  sr = []
  tp = []
  cc = []

  #weather_times= pd.date_range('2010-01-01 00:00:00', '2021-10-01 00:00:00', freq='1H')

  #weather_start_index=
  for i in range(82622,100057):
    for _ in range(4):   #repeat 4 times for the 15 minute intervals in 1 hour
      sr.append(weather_df['surface_solar_radiation (W/m^2)'][i])
      tp.append(weather_df['temperature (degC)'][i])
      cc.append(weather_df['total_cloud_cover (0-1)'][i])

  sr1 = sr
  tp1 = tp
  cc1 = cc

  solar_panel_norm = solar_panel

   # Treating null values
  count=0
  for i,bv in enumerate(solar_panel):
    if(pd.isna(bv)): 
      count+=1
      solar_panel[i]=solar_panel[i-96] # values of previous day

  train_data_x = []
  train_data_y = []
  # lag = 10

  for i in range(solar_panel.shape[0]-lag):
    features = list(solar_panel[i:i+lag])
    
    window_mean = np.mean(solar_panel[i:i+lag])
    window_std = np.std(solar_panel[i:i+lag])

    features.append(times[i+lag].hour//24)
    features.append(times[i+lag].month//24)
    features.append(window_mean)
    features.append(window_std)

    features.append(sr1[i+lag])  #get mean of all solar radiation data in the range
    features.append(tp1[i+lag])  #tempaeratures
    features.append(cc1[i+lag])  #cloud cover
    
  # if(i<10): print(tmp)
    train_data_x.append(np.array(features))
    train_data_y.append(solar_panel[i+lag])

  X = np.array(train_data_x)
  y = np.array(train_data_y)

  X_normed=X/X.mean(axis=0)
  y_normed=y/y.mean(axis=0)
  y_mn=y.mean(axis=0)

  # print(y_mn)
  # print(X_normed.shape)
  # print(y_normed.shape)

  X_train, X_val, y_train, y_val = train_test_split(X_normed, y_normed, test_size=0.2, shuffle=True) # 0.25 x 0.8 = 0.2

  lgb_training_data = lgbm.Dataset(X_train,label=y_train) 
  lgb_valid_data = lgbm.Dataset(X_val,label=y_val,reference=lgb_training_data)

  return lgb_training_data, lgb_valid_data, train_data_x, train_data_y, X_normed, y_normed, y_mn, times

## Predicting a Month Ahead

In [ ]:
def calculatePredictions(building_number, start_index=0, number_of_days_to_predict=31, start_date_str='2020-09-01'):
  lag = 96*30
  
  # lgb_training_data, lgb_valid_data, train_data_x, train_data_y, X_normed, y_normed, y_mn, times = createDatasets(building_number, lag, start_index=0, number_of_days_to_predict=31)
  lgb_training_data, lgb_valid_data, train_data_x, train_data_y, X_normed, y_normed, y_mn, times = createSolarDatasets(building_number, lag, start_index=0, number_of_days_to_predict=31)

  # print(train_data_x)

  model = lgbm.train(params,
                       lgb_training_data,
                       valid_sets=lgb_valid_data,
                       num_boost_round=2000, #5000
                       early_stopping_rounds=100 ,
                       verbose_eval=50)
  
  pred_time_count=4*24*number_of_days_to_predict

  #month_predictions
  pred_times=pd.date_range(start_date_str, periods=pred_time_count, freq='15min')


  pred_y=[]
  pred_y.append(model.predict([train_data_x[-1]])[-1])

  last_lag_y=list(y_normed[-lag+1:])

  last_lag_y.append(pred_y[-1]) # x for first day in month, pred_y[0] is model output

  for i in range(1,pred_time_count):

    features = list(last_lag_y)

    window_mean = np.mean(features)
    window_std = np.std(features)

  # features.append(getTimeBin(pred_times[i]))
    # features.append(pred_times[i].hour)
    # features.append(pred_times[i].dayofweek)
    # features.append(9)#pred_times[i].month)
    # features.append(window_mean)
    # features.append(window_std)

    month=times[i].month

    y_1=model.predict(np.array([np.array(features)]))[0]

    pred_y.append(y_1)
    last_lag_y.pop(0)
    last_lag_y.append(y_1)
    
    #if(i<5): print(features[-10:])

  for ii in range(len(pred_y)): pred_y[ii] *= y_mn

  return pred_y


In [ ]:
# pred_building_1 = calculatePredictions(1, start_index=-22000, number_of_days_to_predict=31)

In [ ]:
starting_indexes = [-22000]*6 + [-40000]*6

In [ ]:
sub_arr = []
sub_indexes = []

for i in range(0,6):
  sub_indexes.append(loaded_data.iloc[i]['series_name'])
  # x = np.random.rand(2880)
  
  # sub_arr.append(x*solar_mean_array[i])

  pred_building_1 = calculatePredictions(1, start_index=starting_indexes[i], number_of_days_to_predict=31)
  
  sub_arr.append(pred_building_1)

sub_arr = np.array(sub_arr)
sub_arr.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in true_divide


Training until validation scores don't improve for 100 rounds.
Did not meet early stopping. Best iteration is:
[2]	valid_0's mape: 0.2943


(1, 2976)

In [ ]:
df_sub = pd.DataFrame(data=sub_arr, index=sub_indexes)

In [ ]:
df_sub.head(15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957,2958,2959,2960,2961,2962,2963,2964,2965,2966,2967,2968,2969,2970,2971,2972,2973,2974,2975
Solar0,10.1685,8.700562,9.3795,10.1685,10.1685,8.700562,9.756,10.1685,10.1685,8.700562,9.4335,10.1685,10.1685,8.700562,9.756,10.1685,10.1685,8.700562,9.4335,10.1685,10.1685,8.700562,9.756,10.1685,10.1685,8.700562,9.4335,10.1685,10.1685,8.700562,9.756,10.1685,10.1685,8.700562,9.4335,10.1685,10.1685,8.700562,9.381,10.1685,...,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756,10.1685,8.700562,9.756,9.756


In [ ]:
df_sub.to_csv("sub.csv",header=False)

In [ ]:
weather_df = pd.read_csv("ERA5_Weather_Data_Monash.csv")

In [ ]:
weather_df.head()

,datetime (UTC),"coordinates (lat,lon)",model (name),model elevation (surface),utc_offset (hrs),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
0,2010-01-01 00:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.26,16.39,2.60,101046.38,0.89,287.01,408.35,1.0
1,2010-01-01 01:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.67,16.29,2.91,101037.96,0.86,360.79,411.02,1.0
2,2010-01-01 02:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.16,15.89,3.26,101017.26,0.87,291.54,410.67,1.0
3,2010-01-01 03:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.46,15.33,3.17,101022.56,0.82,357.11,410.95,1.0
4,2010-01-01 04:00:00,"(-37.91, 145.13)",era5,69.59,10.0,18.53,15.11,2.95,100940.03,0.80,459.91,410.00,0.9
